In [9]:
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

import requests
from pandas.io.json import json_normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [10]:
TargetURL="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [ ]:
#scrape the data
website_url = requests.get(TargetURL).text
website_url

In [ ]:
# use BeautifulSoup to get process on web text
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

In [ ]:
My_table = soup.find('table',{'class':'wikitable sortable'})
print(My_table)

In [ ]:

# use findall to ge the lists includine 'td'
links=My_table.findAll('td')
links

In [15]:
table=np.array(links).reshape((len(links)//3,3))
Table=pd.DataFrame(table,columns=["PostalCode","Borough",'Neighborhood'])
Table.astype(str)
Table.head()

,PostalCode,Borough,Neighborhood
0,<td>M1A</td>,<td>Not assigned</td>,<td>Not assigned </td>
1,<td>M2A</td>,<td>Not assigned</td>,<td>Not assigned </td>
2,<td>M3A</td>,"<td><a href=""/wiki/North_York"" title=""North Yo...","<td><a href=""/wiki/Parkwoods"" title=""Parkwoods..."
3,<td>M4A</td>,"<td><a href=""/wiki/North_York"" title=""North Yo...","<td><a href=""/wiki/Victoria_Village"" title=""Vi..."
4,<td>M5A</td>,"<td><a href=""/wiki/Downtown_Toronto"" title=""Do...","<td><a href=""/wiki/Regent_Park"" title=""Regent ..."


In [16]:
#clean the data
for col in Table.columns:
    Table[col]=Table[col].astype(str).str.replace('<td>','').str.replace('</td>','').str.replace('</a>','').str.replace('\n','').str.replace(']','')
Table.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,"<a href=""/wiki/North_York"" title=""North York"">...","<a href=""/wiki/Parkwoods"" title=""Parkwoods"">Pa..."
3,M4A,"<a href=""/wiki/North_York"" title=""North York"">...","<a href=""/wiki/Victoria_Village"" title=""Victor..."
4,M5A,"<a href=""/wiki/Downtown_Toronto"" title=""Downto...","<a href=""/wiki/Regent_Park"" title=""Regent Park..."


In [17]:
Table['Borough']=Table['Borough'].astype(str).str.split('title=').str[-1].str.split('>').str[-1]
Table['Neighborhood']=Table['Neighborhood'].astype(str).str.split('title=').str[-1].str.split('>').str[-1]
Table.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [18]:
Table.replace('Not assigned',np.nan,inplace=True)
Table.head()

,PostalCode,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [19]:
Table.Neighborhood[8]=Table.Borough[8]

Table.iloc[8,:]

PostalCode      M8A
Borough         NaN
Neighborhood    NaN
Name: 8, dtype: object

In [20]:
Table.dropna(inplace=True)
Table.reset_index(drop=True,inplace=True)
Table.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [21]:
Table=Table.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()
Table.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [22]:
Table.shape

(102, 3)

In [23]:
Table.to_csv('capstone_table1.csv',index=False)